In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

In [2]:
torch.manual_seed(42)

In [3]:
df = pd.read_csv(r"D:\Datasets\fashion_mnist\fashion-mnist_train.csv")
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
#Scaling the features
X_train = X_train/255
X_test = X_test/255

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.get_device_name(0))

True
12.1
NVIDIA GeForce RTX 3070 Laptop GPU


In [8]:
#Creating a custom dataset class

class customDataset(Dataset):    

    def __init__(self, features, labels):
        super().__init__()
        self.features = torch.tensor(features, dtype=torch.float32).reshape(-1, 1, 28, 28)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [9]:
#Creating a train and test datasets
train_dataset = customDataset(X_train, y_train)
test_dataset = customDataset(X_test, y_test)

In [10]:
#Creating train and test dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True)

In [11]:
#Creating an Convolutional Neural Network

class cnn(nn.Module):

    def __init__(self, input_features):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(input_features, 32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(64, 10)
        )

    def forward(self, x):

        x = self.features(x)
        x = self.classifier(x)

        return x

In [17]:
epochs = 50
learning_rate = 0.1

In [18]:
# Instantiating the model
model = cnn(1)
model = model.to(device=device)
# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [19]:
# training loop

for epoch in range(epochs):

    Epoch_loss = 0
    for batch_features, batch_labels in train_loader:

        batch_features, batch_labels = batch_features.to(device=device), batch_labels.to(device=device)
       
        # Forward Pass
        outputs = model(batch_features)

        # Calculate Loss
        loss = criterion(outputs, batch_labels)

        # Backward Pass
        optim.zero_grad()
        loss.backward()

        # Update the weights
        optim.step()
        Epoch_loss += loss.item()

    avg_epoch_loss = Epoch_loss/len(batch_features)
    print(f"Loss after Epoch {epoch+1}: {avg_epoch_loss}")

Loss after Epoch 1: 6.503276036120951
Loss after Epoch 2: 4.355750763555989
Loss after Epoch 3: 3.8523307353025302
Loss after Epoch 4: 3.4049128919141367
Loss after Epoch 5: 3.0858664707047865
Loss after Epoch 6: 2.9008459277101792
Loss after Epoch 7: 2.668069957930129
Loss after Epoch 8: 2.509316243813373
Loss after Epoch 9: 2.3992441134760156
Loss after Epoch 10: 2.184849951096112
Loss after Epoch 11: 2.1287809578934684
Loss after Epoch 12: 1.9754164521582425
Loss after Epoch 13: 1.8911480488604866
Loss after Epoch 14: 1.8722922727756668
Loss after Epoch 15: 1.7346355903573567
Loss after Epoch 16: 1.6743587476667017
Loss after Epoch 17: 1.6256507480575237
Loss after Epoch 18: 1.53984993690392
Loss after Epoch 19: 1.4800222185076564
Loss after Epoch 20: 1.3850797476479784
Loss after Epoch 21: 1.3789085101307137
Loss after Epoch 22: 1.2587957493160502
Loss after Epoch 23: 1.2459907922311686
Loss after Epoch 24: 1.173905657051364
Loss after Epoch 25: 1.2216536363703199
Loss after Epoch 

In [20]:
# Evaluating the model
model.eval()

# Evaluation Code on test data

total = 0
correct = 0

with torch.no_grad():

    for batch_features, batch_labels in test_loader:

        batch_features, batch_labels = batch_features.to(device=device), batch_labels.to(device=device)

        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()

    print(f"Test accuracy: {correct/total}")

Test accuracy: 0.9211666666666667


In [21]:
# Evaluation Code on test data

total_train = 0
correct_train = 0

with torch.no_grad():

    for batch_features, batch_labels in train_loader:

        batch_features, batch_labels = batch_features.to(device=device), batch_labels.to(device=device)

        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        
        total_train += batch_labels.shape[0]
        correct_train += (predicted == batch_labels).sum().item()

    print(f"Train accuracy: {correct_train/total_train}")

Train accuracy: 0.9918333333333333
